In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
files_path = "/users/PAS1937/battula12/Shanmukh/Projects/MVTSS/Controlled_Experiments/Logs/multi_task/moving/"

array_types = ["linear_array", "circular_array"]
dataset_conditions = ["reverb", "noisy_reverb"]
noise_simulations = ["point_source", "diffuse"]

In [3]:
search_pattern = ["acc_epoch"]

array_type = "circular_array"
reverb_log_dir = os.path.join(files_path, f"{array_type}/reverb")
noisy_reverb_ps_log_dir = os.path.join(files_path, f"{array_type}/noisy_reverb/point_source")
noisy_reverb_diffuse_log_dir = os.path.join(files_path, f"{array_type}/noisy_reverb/diffuse")


In [4]:
def get_test_config_details(line):
    snr,t60,app_str = None, None,""
    
    if "dataset_condition:" in line:
        line_split_info = line.split(',')
        if line_split_info[-1]=='':
            line_split_info.pop()
        dataset_condition = line_split_info[-1].split()[-1]
        if dataset_condition=="noisy":
            snr = line_split_info[-3].split()[-1]
            app_str = f'snr_{snr}'
        elif dataset_condition=="noisy_reverb":
            
            snr = line_split_info[-3].split()[-1]
            t60 = line_split_info[-4].split()[-1]
            app_str = f'snr_{snr}_t60_{t60}'
            
        elif dataset_condition=="reverb":
            t60 = line_split_info[-4].split()[-1]
            app_str = f't60_{t60}'

            
    return snr, t60, app_str

In [7]:
def search_pattern_file(file_path, search_pattern_list):
    exp_dict = {}
    app_str = ""
    with open(file_path, 'r') as f:
        for line in f.readlines():
            line = line.strip()
            if app_str=="":
                snr, t60, app_str = get_test_config_details(line)
                
            for search_pattern in search_pattern_list: #search_doa_pattern_list:
                if search_pattern in line:
                    line_split_info = line.split()
                    print(line_split_info, len(line_split_info))
                    if len(line_split_info)==5:
                        key_str = f'{search_pattern}_{app_str}'
                        exp_dict[key_str] = float(line_split_info[3])
                        #print(key_str, line.split()[3])
                
    return exp_dict

In [5]:
def get_all_exp_dict(log_dir: str, search_pattern: list):
    log_dir_files = os.listdir(log_dir)
    log_dir_files = [x for x in log_dir_files if os.path.isfile(os.path.join(log_dir,x))]
    #print(log_dir_files)
    _exp_dict={}
    for _file in log_dir_files:
        file_path = os.path.join(log_dir, _file)
        #print(file_path)
        exp_dict = search_pattern_file(file_path, search_pattern)
        _exp_dict.update(exp_dict)
    
    return _exp_dict

In [8]:
reverb_exp_dict = get_all_exp_dict(reverb_log_dir, search_pattern)
noisy_reverb_ps_exp_dict = get_all_exp_dict(noisy_reverb_ps_log_dir, search_pattern)
noisy_reverb_diffuse_exp_dict = get_all_exp_dict(noisy_reverb_diffuse_log_dir, search_pattern)

['│', 'acc_epoch', '│', '0.9986311197280884', '│'] 5
['│', 'acc_epoch', '│', '0.9991754293441772', '│'] 5
['│', 'acc_epoch', '│', '0.9988487362861633', '│'] 5
['│', 'acc_epoch', '│', '0.9984169006347656', '│'] 5
['│', 'acc_epoch', '│', '0.9976455569267273', '│'] 5
['│', 'acc_epoch', '│', '0.9959070086479187', '│'] 5
['│', 'acc_epoch', '│', '0.9936320185661316', '│'] 5
['│', 'acc_epoch', '│', '0.9810401797294617', '│'] 5
['│', 'acc_epoch', '│', '0.9920080900192261', '│'] 5
['│', 'acc_epoch', '│', '0.947060763835907', '│'] 5
['│', 'acc_epoch', '│', '0.9013878107070923', '│'] 5
['│', 'acc_epoch', '│', '0.9902311563491821', '│'] 5
['│', 'acc_epoch', '│', '0.9642425775527954', '│'] 5
['│', 'acc_epoch', '│', '0.9751957654953003', '│'] 5
['│', 'acc_epoch', '│', '0.9261616468429565', '│'] 5
['│', 'acc_epoch', '│', '0.9928839802742004', '│'] 5
['│', 'acc_epoch', '│', '0.9753925204277039', '│'] 5
['│', 'acc_epoch', '│', '0.9900338053703308', '│'] 5
['│', 'acc_epoch', '│', '0.9968994855880737', '

In [9]:
reverb_exp_dict

{'acc_epoch_t60_0.0': 0.9986311197280884,
 'acc_epoch_t60_0.2': 0.9991754293441772,
 'acc_epoch_t60_0.4': 0.9988487362861633,
 'acc_epoch_t60_0.6': 0.9984169006347656,
 'acc_epoch_t60_0.8': 0.9976455569267273,
 'acc_epoch_t60_1.0': 0.9959070086479187}

In [10]:
#plotting frm Acc
from itertools import product

noisy_snr_list = [-5.0, -2.5, 0.0, 2.5, 5.0, 10.0, 15.0, 20.0]

noisy_reverb_snr_list = [-5.0, 0.0, 5.0]#, 10.0, 20.0 ]
noisy_reverb_t60_list = [0.2, 0.4, 0.6, 0.8, 1.0]
noisy_reverb_list = list(product(noisy_reverb_snr_list, noisy_reverb_t60_list))

#reverb_t60_list = [0.0, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
reverb_t60_list = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]

In [11]:
def build_app_str(dataset_condition, val):
    if dataset_condition=="noisy":
        app_str = f'snr_{val}'
    elif dataset_condition=="noisy_reverb":
        app_str = f'snr_{val[0]}_t60_{val[1]}'   
    elif dataset_condition=="reverb":
        app_str = f't60_{val}'
    return app_str

In [12]:
def _print(measure, exp_dict, dataset_condition, tst_conditions, loss_list, dataset_simulation=""):
    print(tst_conditions)
    for loss in loss_list:
        val_list = []
        for tst_val in tst_conditions:
            app_str = build_app_str(dataset_condition, tst_val)
            key_str = f'{measure}_{app_str}'

            val_list.append(exp_dict[key_str])
        print(loss, val_list)

In [13]:
_print("acc_epoch", reverb_exp_dict, "reverb", reverb_t60_list, [""], "")
_print("acc_epoch", noisy_reverb_ps_exp_dict, "noisy_reverb", noisy_reverb_list, [""], "ps")
_print("acc_epoch", noisy_reverb_diffuse_exp_dict, "noisy_reverb", noisy_reverb_list, [""], "diffuse")

[0.0, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]


KeyError: 'acc_epoch_t60_0.3'

In [63]:
noisy_reverb_ps_exp_dict

{'acc_epoch_snr_-5.0_t60_0.4': 0.9509741067886353,
 'acc_epoch_snr_-5.0_t60_0.2': 0.9200452566146851,
 'acc_epoch_snr_5.0_t60_0.6': 0.9968636631965637,
 'acc_epoch_snr_5.0_t60_0.8': 0.9954746961593628,
 'acc_epoch_snr_-5.0_t60_0.6': 0.9367882609367371,
 'acc_epoch_snr_-5.0_t60_0.8': 0.9133656620979309,
 'acc_epoch_snr_0.0_t60_0.6': 0.9911887049674988,
 'acc_epoch_snr_0.0_t60_0.4': 0.9942905306816101,
 'acc_epoch_snr_5.0_t60_1.0': 0.9934441447257996,
 'acc_epoch_snr_-5.0_t60_1.0': 0.8878486752510071,
 'acc_epoch_snr_0.0_t60_0.2': 0.9814140200614929,
 'acc_epoch_snr_5.0_t60_0.4': 0.9978854060173035,
 'acc_epoch_snr_5.0_t60_0.2': 0.9939332008361816,
 'acc_epoch_snr_0.0_t60_0.8': 0.9876143336296082,
 'acc_epoch_snr_0.0_t60_1.0': 0.9830060005187988}

In [7]:
se_measures_pattern = ["STOI", "PESQ_NB"]

In [22]:
reverb_exp_dict = get_all_exp_dict(reverb_log_dir, se_measures_pattern)
noisy_reverb_ps_exp_dict = get_all_exp_dict(noisy_reverb_ps_log_dir, se_measures_pattern)
noisy_reverb_diffuse_exp_dict = get_all_exp_dict(noisy_reverb_diffuse_log_dir, se_measures_pattern)

['│', 'ESTOI_epoch', '│', '0.5490276977089484', '│'] 5
['│', 'MIX_ESTOI_epoch', '│', '0.23888022043295973', '│'] 5
['│', 'MIX_PESQ_NB_epoch', '│', '1.2094317638499175', '│'] 5
['│', 'MIX_STOI_epoch', '│', '0.5238080213259423', '│'] 5
['│', 'PESQ_NB_epoch', '│', '1.9544265247621588', '│'] 5
['│', 'STOI_epoch', '│', '0.7538456853848726', '│'] 5
['│', 'ESTOI_epoch', '│', '0.49865645102701883', '│'] 5
['│', 'MIX_ESTOI_epoch', '│', '0.20382957277911545', '│'] 5
['│', 'MIX_PESQ_NB_epoch', '│', '1.1747029425559843', '│'] 5
['│', 'MIX_STOI_epoch', '│', '0.492614943755469', '│'] 5
['│', 'PESQ_NB_epoch', '│', '1.8065288900485403', '│'] 5
['│', 'STOI_epoch', '│', '0.7180457805808883', '│'] 5
['│', 'ESTOI_epoch', '│', '0.6399418776328593', '│'] 5
['│', 'MIX_ESTOI_epoch', '│', '0.2786667285879045', '│'] 5
['│', 'MIX_PESQ_NB_epoch', '│', '1.267096366337059', '│'] 5
['│', 'MIX_STOI_epoch', '│', '0.5626695231263328', '│'] 5
['│', 'PESQ_NB_epoch', '│', '2.235184134414883', '│'] 5
['│', 'STOI_epoch', '│

In [27]:
noisy_reverb_ps_exp_dict

{'STOI_snr_-5.0_t60_0.4': 0.7538456853848726,
 'PESQ_NB_snr_-5.0_t60_0.4': 1.9544265247621588,
 'STOI_snr_-5.0_t60_0.6': 0.7180457805808883,
 'PESQ_NB_snr_-5.0_t60_0.6': 1.8065288900485403,
 'STOI_snr_-5.0_t60_0.2': 0.8144112745121372,
 'PESQ_NB_snr_-5.0_t60_0.2': 2.235184134414883,
 'STOI_snr_-5.0_t60_0.8': 0.6896502858103852,
 'PESQ_NB_snr_-5.0_t60_0.8': 1.7074318411886142,
 'STOI_snr_-5.0_t60_1.0': 0.6653000964445419,
 'PESQ_NB_snr_-5.0_t60_1.0': 1.6336026350859807,
 'STOI_snr_0.0_t60_0.2': 0.8912287365184997,
 'PESQ_NB_snr_0.0_t60_0.2': 2.686611786204634,
 'STOI_snr_0.0_t60_0.4': 0.8460506305682103,
 'PESQ_NB_snr_0.0_t60_0.4': 2.393241406091005,
 'STOI_snr_0.0_t60_0.6': 0.8162416709817713,
 'PESQ_NB_snr_0.0_t60_0.6': 2.2276321429771047,
 'STOI_snr_0.0_t60_0.8': 0.791171740315867,
 'PESQ_NB_snr_0.0_t60_0.8': 2.108664427042217,
 'STOI_snr_0.0_t60_1.0': 0.7689680643061735,
 'PESQ_NB_snr_0.0_t60_1.0': 2.0185260102724176,
 'STOI_snr_5.0_t60_0.2': 0.9323986994777643,
 'PESQ_NB_snr_5.0_t6

In [24]:
t60_list = [0.0, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
measures_list = ["STOI", "PESQ_NB"]
#Loss = ["MIX", "MISO_RI", "MISO_RI_MAG"]
keys_list = reverb_exp_dict.keys()

for measure in measures_list:
    val_list=[]
    for t60 in t60_list: 
        app_str = f"t60_{t60}"  
        key_str = f'{measure}_{app_str}' #if "MIX" not in loss else f'{loss}_{measure}_epoch_{app_str}'  #f'{loss}_est_{num_mics}mic_enh_{measure}_{app_str}' #inside file pattern
        if key_str in keys_list:
            val = reverb_exp_dict[key_str]
        else:
            val = 0.0
        val_list.append(val)
    print(f" {measure}, {np.mean(val_list)}")

 STOI, 0.0
 PESQ_NB, 0.0


In [28]:
exp_dict = noisy_reverb_ps_exp_dict # noisy_reverb_diffuse_exp_dict

snr_list = [-5.0, 0.0, 5.0]
t60_list = [0.2, 0.4, 0.6, 0.8, 1.0]
measures_list = ["STOI", "PESQ_NB"]
keys_list = exp_dict.keys()



for measure in measures_list:
    val_list=[] 
    for _snr in snr_list:
        for t60 in t60_list:                                                   
                            
            app_str = f"snr_{_snr}_t60_{t60}"  
            key_str = f'{measure}_{app_str}' #if "MIX" not in loss else f'{loss}_{measure}_epoch_{app_str}' #f'{measure}_{loss}_est_{num_mics}mic_enh_{mic_idx}_{app_str}' #inside file pattern

            if key_str in keys_list:
                val = exp_dict[key_str]
            else:
                #print(key_str)
                val = 0.0
            val_list.append(val)
    #print(val_list)
    print(f"{measure}, {np.mean(val_list)}")

STOI, 0.8054322208898302
PESQ_NB, 2.190142962604347


Circular array
nr_ps:  STOI, 0.8334184718703059 , PESQ_NB, 2.335496293759587 \\
nr_dif:  STOI, 0.8284373132143797, PESQ_NB, 2.281116524542644